In [1]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq
 
from bokeh.palettes import brewer, Spectral6
from bokeh.io import show, output_file, curdoc
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, Panel, Tabs, GeoJSONDataSource, LinearColorMapper
from bokeh.models import ColorBar
from bokeh.layouts import column, gridplot, row
from bokeh.transform import factor_cmap
from bokeh.models import NumeralTickFormatter, Title, Label, Paragraph, Div, CustomJSHover, BoxAnnotation

This file creates the aggregate trade data.


In [2]:
trade_type = "imports"

my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"
# This is my key. I'm nice and I have it posted. If you will be doing more with this
# please get your own key!

In [3]:
crl = ["darkblue","slategray","crimson"]

background = "#ffffff"

In [4]:
last_month = 7

---
#### Grabe the trade data using the Census's API

In [5]:
def census_trade(url, trade_type, country, product_level):
    
    r = requests.get(url) 
    
    print(r)
    
    df = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

    df.columns = r.json()[0]

    df.time = pd.to_datetime(df.time, format="%Y-%m")
    # This is so I can call this correctly...
    
    if trade_type == "imports":
        
        trade_label = country + "_" + trade_type
        
        df[trade_label] = df["CON_VAL_MO"].astype(float)
        
        df[product_level] = df["NAICS"].astype(str)
        
        df.drop(["CON_VAL_MO", "NAICS", "COMM_LVL"], axis = 1, inplace = True)
        
    if trade_type == "exports":
    
        trade_label = country + "_" + trade_type
        
        df[trade_label] = df["ALL_VAL_MO"].astype(float)

        df[product_level] = df["NAICS"].astype(str)
        
        df.drop(["ALL_VAL_MO", "NAICS", "COMM_LVL"], axis = 1, inplace = True)
        
    if trade_type == "country_exports":
    
        trade_label = "total_exports"
        
        df[trade_label] = df["ALL_VAL_MO"].astype(float)
        
        df = df[df.SUMMARY_LVL == "DET"]
        
        df.drop(["ALL_VAL_MO", "CTY_CODE", "SUMMARY_LVL"], axis = 1, inplace = True)
        
        df = df[df.CTY_NAME != "TOTAL FOR ALL COUNTRIES"]
        
    if trade_type == "country_imports":
    
        trade_label = "total_imports"
        
        df[trade_label] = df["CON_VAL_MO"].astype(float)
        
        df = df[df.SUMMARY_LVL == "DET"]
        
        df.drop(["CON_VAL_MO", "CTY_CODE", "SUMMARY_LVL"], axis = 1, inplace = True)
        
        
    
    return df

---
### And this is the Product Plot

In [6]:
end_use = "naics?get=NAICS,CON_VAL_MO,NAICS_LDESC"

url = "https://api.census.gov/data/timeseries/intltrade/imports/" + end_use 
url = url + my_key + "&time==from+2013-01" + "&COMM_LVL=NA3"

url = url

df = census_trade(url, trade_type, "total", "naics")

df.time.max()

<Response [200]>


Timestamp('2020-05-01 00:00:00')

In [7]:
df.head()

,NAICS_LDESC,time,total_imports,naics
0,"FABRICATED METAL PRODUCTS, NESOI",2013-01-01,4.973810e+09,332
1,"FABRICATED METAL PRODUCTS, NESOI",2013-02-01,4.688597e+09,332
2,"FABRICATED METAL PRODUCTS, NESOI",2013-03-01,4.752188e+09,332
3,"FABRICATED METAL PRODUCTS, NESOI",2013-04-01,5.126111e+09,332
4,"FABRICATED METAL PRODUCTS, NESOI",2013-05-01,5.328913e+09,332


In [8]:
#end_use = "hs?get=E_COMMODITY,ALL_VAL_MO,E_COMMODITY_SDESC"

#url = "https://api.census.gov/data/timeseries/intltrade/exports/" + end_use 
#url = url + my_key + "&time==from+2013-01" + "&COMM_LVL=HS2"

#url = url

#df = census_trade(url, trade_type, "total", "hs2")

#df.time.max()

In [9]:
def growth_trade(foo):
    # what this function does is take a dataframe and create a relative 
    
    foo["growth"] = 100*((foo.total_imports/foo.total_imports.shift(12)) - 1)
    
    return foo

In [10]:
def make_trade_time(df):
        
    foo = df.drop(labels = ["naics"], axis = 1).reset_index().pivot(index = "time", columns = "NAICS_LDESC")
    
    foo.drop(labels = ["index"], axis = 1, inplace = True)
    
    return foo

In [11]:
grp = df.groupby(["naics"])

top_products = grp.agg({"total_imports":"sum"})

plist = list(top_products.sort_values(by = "total_imports", ascending = False).index.values)[0:9]

df = df[df.naics.isin(plist)]

In [12]:
grp = df.groupby(["naics"])

productdf = grp.apply(growth_trade)

In [30]:
def make_time_by_product(df, level):

    height = 533
    width = 625

    foobar = make_trade_time(df)
    
    if level == "total_imports":
        title = "By Product (NAICS 3), $ US Imports "
    
    if level == "growth":
        title = "By Product (NAICS 3), % Year over Year Change in US Imports"
    
    p = figure(plot_height=height, plot_width = width, x_axis_type="datetime",toolbar_location = 'below',
           tools = "box_zoom, reset, pan, xwheel_zoom", title = title, x_range = (dt.datetime(2019,1,1),dt.datetime(2020,last_month,1)) ) 

    numlines=len(foobar[level].columns)

    multi_line_source = ColumnDataSource({
        'xs': [foobar.index.values]*numlines,
        'ys': [foobar[level, name].values for name in foobar[level]],
        'label': [name for name in foobar["total_imports"].columns]})

    p.multi_line(xs= "xs",
                ys= "ys",
                line_width=2.5, line_alpha=0.45, line_color = "slategray",
                 hover_line_alpha=0.75, hover_line_width = 5,
                hover_line_color= "crimson", source = multi_line_source)
    
    y_custom = CustomJSHover(code=""" return '' + special_vars.data_y
            """)
            
    TIMETOOLTIPS = """
            <div style="background-color:#F5F5F5; opacity: 0.95; border: 5px 5px 5px 5px;">
            <div style = "text-align:left;">
            <span style="font-size: 13px; font-weight: bold"> @label
             </span>
             </div>
             <div style = "text-align:left;">"""
    
    if level == "total_imports":
        TIMETOOLTIPS = TIMETOOLTIPS + """
            <span style="font-size: 13px; font-weight: bold"> $data_x{%b %Y}:  $data_y{$0.0a}</span>   
            </div>
            </div>
            """
    if level == "growth":
        TIMETOOLTIPS = TIMETOOLTIPS + """
            <span style="font-size: 13px; font-weight: bold"> $data_x{%b %Y}:  $data_y{0.0}%</span>   
            </div>
            </div>
            """

    p.add_tools(HoverTool(tooltips = TIMETOOLTIPS,  line_policy='nearest', formatters={'$data_x': 'datetime'}))
    p.title.text_font_size = '13pt'
    p.background_fill_color = background 
    p.background_fill_alpha = 0.75
    p.border_fill_color = background 
    
    #tradewar_box = BoxAnnotation(left=dt.datetime(2018,7,1), right=dt.datetime(2019,10,11), fill_color='red', fill_alpha=0.1)
    #p.add_layout(tradewar_box)
    
    pandemic_box = BoxAnnotation(left=dt.datetime(2020,3,1), right=dt.datetime(2020,12,31), fill_color='red', fill_alpha=0.1)
    p.add_layout(pandemic_box)
    
    #p.yaxis.axis_label = 
    p.yaxis.axis_label_text_font_style = 'bold'
    p.yaxis.axis_label_text_font_size = "13px"

    p.yaxis.minor_tick_line_color = None
    #p.y_range.start = 0
    
    #p.x_range.max_interval = dt.datetime(2020,6,1)
    
    #description = """Each line is a major product catagory in Annex 6-1 of The Agreement."""
    #p.add_layout(Title(text=description, text_font_style="normal", text_font_size="9pt"), 'below')
    #description = "Red marks the period of Section 301 tariffs and retaliation. Blue is period of agreement."""
    #p.add_layout(Title(text=description, text_font_style="normal", text_font_size="9pt"), 'below')
    
    div0 = Div(text = """Each line is a 3 digit NAICS catagory. Only the top 10 catagories are shown. Red is the start of the NBER
    determined recession and Covid-19 pandemic in the US.""", width=555, background = background )
    
    if level == "total_imports":
        p.yaxis.formatter = NumeralTickFormatter(format="($0. a)")
    
    p.outline_line_color = None
    
    p.toolbar.active_drag = None
    p.toolbar.autohide = True
    
    p.sizing_mode= "scale_both"
    p.max_height = height
    p.max_width = width
    p.min_border_left = 0
    return p

---
### This is the Country plot...

In [14]:
end_use = "naics?get=CON_VAL_MO,CTY_CODE,CTY_NAME,SUMMARY_LVL"

url = "https://api.census.gov/data/timeseries/intltrade/imports/" + end_use 
url = url + my_key + "&time==from+2013-01"

url = url

df = census_trade(url,"country_imports","foo", "foo")

df.time.max()

<Response [200]>


Timestamp('2020-05-01 00:00:00')

In [15]:
grp = df.groupby(["CTY_NAME"])

top_products = grp.agg({"total_imports":"sum"})

clist = list(top_products.sort_values(by = "total_imports", ascending = False).index.values)[0:11]

df = df[df.CTY_NAME.isin(clist)]

In [16]:
grp = df.groupby(["CTY_NAME"])

country_df = grp.apply(growth_trade)

In [17]:
country_df.head()

,CTY_NAME,time,total_imports,growth
2,UNITED KINGDOM,2013-01-01,3.917330e+09,NaN
9,FRANCE,2013-01-01,3.123040e+09,NaN
10,GERMANY,2013-01-01,7.672833e+09,NaN
28,TOTAL FOR ALL COUNTRIES,2013-01-01,1.824581e+11,NaN
43,CANADA,2013-01-01,2.792022e+10,NaN


In [18]:
def make_trade_time_country(df):
        
    foo = df.reset_index().pivot(index = "time", columns = "CTY_NAME")
    
    foo.drop(labels = ["index"], axis = 1, inplace = True)
    
    return foo

In [19]:
test = make_trade_time_country(country_df)

In [20]:
test["total_imports"].drop([ "TOTAL FOR ALL COUNTRIES"], axis = 1)

CTY_NAME,CANADA,CHINA,FRANCE,GERMANY,INDIA,ITALY,JAPAN,"KOREA, SOUTH",MEXICO,UNITED KINGDOM
time,,,,,,,,,,
2013-01-01,2.792022e+10,3.704520e+10,3.123040e+09,7.672833e+09,3.102502e+09,2.848031e+09,1.124743e+10,5.390157e+09,2.110684e+10,3.917330e+09
2013-02-01,2.583651e+10,3.258536e+10,3.074257e+09,8.216054e+09,2.809179e+09,2.716720e+09,1.096638e+10,4.544637e+09,2.152986e+10,3.871755e+09
2013-03-01,2.830020e+10,2.754229e+10,3.919916e+09,9.554447e+09,3.657293e+09,3.149917e+09,1.225345e+10,5.177314e+09,2.294209e+10,4.563933e+09
2013-04-01,2.879433e+10,3.303855e+10,3.922591e+09,9.403480e+09,3.993297e+09,3.255985e+09,1.190768e+10,5.498595e+09,2.336942e+10,4.237256e+09
2013-05-01,2.842892e+10,3.645417e+10,3.699728e+09,9.491015e+09,4.196371e+09,3.246507e+09,1.107665e+10,5.600834e+09,2.458872e+10,4.502089e+09
...,...,...,...,...,...,...,...,...,...,...
2020-01-01,2.534355e+10,3.346214e+10,3.832416e+09,9.918148e+09,5.143155e+09,4.246087e+09,1.065182e+10,6.230719e+09,2.851593e+10,4.541208e+09
2020-02-01,2.426626e+10,2.343340e+10,3.657078e+09,8.946864e+09,4.539631e+09,3.965939e+09,1.064309e+10,5.490279e+09,2.898957e+10,4.495829e+09
2020-03-01,2.512440e+10,1.973310e+10,4.388439e+09,1.180746e+10,4.606066e+09,4.602397e+09,1.194705e+10,6.726884e+09,2.989388e+10,4.990931e+09


In [29]:
def make_time_by_country(df, level):

    height = 533
    width = 625

    foobar = make_trade_time_country(df)
    
    if level == "total_imports":
        title = "By Source Country, $ US Imports "
        
        foobar.drop([ "TOTAL FOR ALL COUNTRIES"], axis = 1, level = 1, inplace = True)
    
    if level == "growth":
        title = "By Source Country, % Year over Year Change in US Imports "
    
    p = figure(plot_height=height, plot_width = width, x_axis_type="datetime",toolbar_location = 'below',
           tools = "box_zoom, reset, pan, xwheel_zoom", title = title, x_range = (dt.datetime(2019,1,1),dt.datetime(2020,last_month,1)) ) 

    numlines=len(foobar[level].columns)

    multi_line_source = ColumnDataSource({
        'xs': [foobar.index.values]*numlines,
        'ys': [foobar[level, name].values for name in foobar[level]],
        'label': [name for name in foobar["total_imports"].columns]})

    p.multi_line(xs= "xs",
                ys= "ys",
                line_width=2.5, line_alpha=0.45, line_color = "slategray",
                 hover_line_alpha=0.75, hover_line_width = 5,
                hover_line_color= "crimson", source = multi_line_source)
    
    y_custom = CustomJSHover(code=""" return '' + special_vars.data_y
            """)
            
    TIMETOOLTIPS = """
            <div style="background-color:#F5F5F5; opacity: 0.95; border: 5px 5px 5px 5px;">
            <div style = "text-align:left;">
            <span style="font-size: 13px; font-weight: bold"> @label
             </span>
             </div>
             <div style = "text-align:left;">"""
    
    if level == "total_imports":
        TIMETOOLTIPS = TIMETOOLTIPS + """
            <span style="font-size: 13px; font-weight: bold"> $data_x{%b %Y}:  $data_y{$0.0a}</span>   
            </div>
            </div>
            """
    if level == "growth":
        TIMETOOLTIPS = TIMETOOLTIPS + """
            <span style="font-size: 13px; font-weight: bold"> $data_x{%b %Y}:  $data_y{0.0}%</span>   
            </div>
            </div>
            """

    p.add_tools(HoverTool(tooltips = TIMETOOLTIPS,  line_policy='nearest', formatters={'$data_x': 'datetime'}))
    p.title.text_font_size = '13pt'
    p.background_fill_color = background 
    p.background_fill_alpha = 0.75
    p.border_fill_color = background 
    
    
    pandemic_box = BoxAnnotation(left=dt.datetime(2020,3,1), right=dt.datetime(2020,12,31), fill_color='red', fill_alpha=0.1)
    p.add_layout(pandemic_box)
    
    #p.yaxis.axis_label = 
    p.yaxis.axis_label_text_font_style = 'bold'
    p.yaxis.axis_label_text_font_size = "13px"

    p.yaxis.minor_tick_line_color = None
    
    if level == "total_imports":
        p.yaxis.formatter = NumeralTickFormatter(format="($0. a)")
    
    p.outline_line_color = None
    p.sizing_mode= "scale_both"
    p.max_height = height
    p.max_width = width
    
    p.toolbar.active_drag = None
    p.toolbar.autohide = True
    p.min_border_left = 0

    return p

In [31]:
def make_panel_fig():
    
    plevel = make_time_by_product(productdf, "total_imports")
    
    pchange = make_time_by_product(productdf, "growth")
    
    #################################################################33
    
    clevel = make_time_by_country(country_df, "total_imports")
    
    cchange = make_time_by_country(country_df, "growth")
    
    #################################################################
    
    tab1 = Panel(child= plevel, title="Imports in $")

    tab2 = Panel(child= pchange, title="Imports as % YoY")
        
    tab3 = Panel(child= clevel, title="Imports in $")

    tab4 = Panel(child= cchange, title="Imports as % YoY")
        
    outfig = row(Tabs(tabs=[tab4,  tab3], tabs_location = "above"), Tabs(tabs=[tab2,  tab1], tabs_location = "above"))
    
    return outfig

In [32]:
output_file('.\\docs\\' + "us_imports.html")

outfig = make_panel_fig()

show(outfig)